In [1]:
from datasets import load_dataset, Dataset

/home/faith/pythonenvs/nlpia_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_dataset = load_dataset("lewtun/github-issues",split='train')

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
filtered_dataset = raw_dataset.filter(lambda x : (x['is_pull_request'] == False and len(x['comments']) > 0))

In [4]:
filtered_dataset.features

{'url': Value(dtype='string', id=None),
 'repository_url': Value(dtype='string', id=None),
 'labels_url': Value(dtype='string', id=None),
 'comments_url': Value(dtype='string', id=None),
 'events_url': Value(dtype='string', id=None),
 'html_url': Value(dtype='string', id=None),
 'id': Value(dtype='int64', id=None),
 'node_id': Value(dtype='string', id=None),
 'number': Value(dtype='int64', id=None),
 'title': Value(dtype='string', id=None),
 'user': {'login': Value(dtype='string', id=None),
  'id': Value(dtype='int64', id=None),
  'node_id': Value(dtype='string', id=None),
  'avatar_url': Value(dtype='string', id=None),
  'gravatar_id': Value(dtype='string', id=None),
  'url': Value(dtype='string', id=None),
  'html_url': Value(dtype='string', id=None),
  'followers_url': Value(dtype='string', id=None),
  'following_url': Value(dtype='string', id=None),
  'gists_url': Value(dtype='string', id=None),
  'starred_url': Value(dtype='string', id=None),
  'subscriptions_url': Value(dtype='st

In [5]:
final_df = filtered_dataset.remove_columns(set(['title','body','comments','html_url']).symmetric_difference(filtered_dataset.column_names))

In [6]:
final_df.set_format('pandas')

In [7]:
final_df = final_df[:]

In [8]:
final_df = final_df.explode(column='comments',ignore_index=True)
final_df[0:10]

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...
4,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Well it can cause issue with anyone that updat...,## Describe the bug\r\nAfter upgrading to data...
5,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"I just merged a fix, let me know if you're sti...",## Describe the bug\r\nAfter upgrading to data...
6,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Definitely works on several manual cases with ...,## Describe the bug\r\nAfter upgrading to data...
7,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Fixed by #2947.,## Describe the bug\r\nAfter upgrading to data...
8,https://github.com/huggingface/datasets/issues...,OSCAR unshuffled_original_ko: NonMatchingSplit...,I tried `unshuffled_original_da` and it is als...,## Describe the bug\r\n\r\nCannot download OSC...
9,https://github.com/huggingface/datasets/issues...,load_dataset using default cache on Windows ca...,"Hi @daqieq, thanks for reporting.\r\n\r\nUnfor...",## Describe the bug\r\nStandard process to dow...


In [9]:
final_dataset = Dataset.from_pandas(final_df)

In [10]:
final_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [11]:
final_dataset = final_dataset.map(lambda x : {"comment_length" : len(x['comments'].split())})

Map: 100%|██████████| 2964/2964 [00:00<00:00, 9302.00 examples/s]


In [12]:
final_dataset = final_dataset.filter(lambda x : x['comment_length'] > 10)

Filter: 100%|██████████| 2964/2964 [00:00<00:00, 67964.06 examples/s]


In [13]:
final_dataset = final_dataset.map(lambda x : {"text": "[TITLE] "+ x['title'] + " [BODY] " + x['body'] + " [COMMENTS] "+ x['comments']})

Map: 100%|██████████| 2478/2478 [00:00<00:00, 3855.66 examples/s]


In [14]:
final_dataset[0]

{'html_url': 'https://github.com/huggingface/datasets/issues/2945',
 'title': 'Protect master branch',
 'comments': '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).',
 'body': 'After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pul

In [15]:
from transformers import AutoTokenizer, AutoModel

In [16]:
checkpoint = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

In [17]:
model = AutoModel.from_pretrained(checkpoint)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [18]:
import torch
torch.set_default_device('cuda')

In [19]:
model.to(torch.get_default_device())

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [ ]:
def cls_pooling(model_output):
    # print(model_output.last_hidden_state[:,0,:].shape)
    return model_output.last_hidden_state[:, 0]

In [68]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(torch.get_default_device()) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # print(model_output.last_hidden_state.shape)
    return cls_pooling(model_output)

In [69]:
embedding = get_embeddings(final_dataset[0]["comments"])

torch.Size([1, 768])


In [70]:
embedding.shape

torch.Size([1, 768])

In [24]:
embeddings_dataset = final_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map: 100%|██████████| 2478/2478 [01:58<00:00, 20.99 examples/s]


In [27]:
embeddings_dataset.add_faiss_index(column="embeddings")

100%|██████████| 3/3 [00:00<00:00, 222.74it/s]


Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2478
})

In [28]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [29]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [30]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [31]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

COMMENT: Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if offline mode is added similar to how `transformers` loads models offline fine.

@mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could you please elaborate on how that should look like?
SCORE: 27.256507873535156
TITLE: Discussion using datasets in offline mode
URL: https://github.com/huggingface/datasets/issues/824

COMMENT: I opened a PR that allows to reload modules that have already been loaded once even if there's no internet.

Let me know if you know other ways that can make the offline mode experience better. I'd be happy to add them :) 

I already note the "freeze" modules option, to prevent local modules updates. It would be a cool feature.

----------

> @mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Co